In [1]:
# Helper functions
from pathlib import Path
from copy import deepcopy
from typing import Tuple, List, Dict

import numpy as np
import open3d as o3d
from transforms3d.quaternions import quat2mat, mat2quat

import rosbag
from cv_bridge import CvBridge
from geometry_msgs.msg import Transform

def transform_msg2mat(transform: Transform) -> np.ndarray:
    """Convert a geometry_msgs/Transform to a 4x4 transformation matrix
    :param transform: a geometry_msgs.msg.Transform.
    """
    p = transform.translation
    q = transform.rotation
    
    T = np.eye(4)
    T[:3, :3] = quat2mat([q.w, q.x, q.y, q.z])  # [w, x, y, z]
    T[:3, 3] = [p.x, p.y, p.z]
    return T

def read_frame_data_from_bag(bag: rosbag.Bag) -> Dict[str, list]:
    """Read frame data from a rosbag"""
    frame_data = {
        'rgb_imgs': [],  # [H, W, 3] uint8 np.ndarray
        'rgb_img_timestamps': [],
        'rgb_intrinsics': [],  # [3, 3] float64 np.ndarray
        'rgb_intr_timestamps': [],
        'depth_imgs': [],  # [H, W] uint16 np.ndarray
        'depth_img_timestamps': [],
        'depth_intrinsics': [],  # [3, 3] float64 np.ndarray
        'depth_intr_timestamps': [],
        'poses': [],  # T_world_cam, [4, 4] float64 np.ndarray
        'pose_timestamps': [],
        'timestamps': [],  # unique timestamps with imgs and poses
    }
    
    cvbridge = CvBridge()
    topic_msgs = bag.read_messages(topics=['/tf',
                                           '/camera/color/image_raw',
                                           '/camera/color/camera_info',
                                           '/camera/aligned_depth_to_color/image_raw',
                                           '/camera/aligned_depth_to_color/camera_info'])
    for i, (topic, msg, t) in enumerate(topic_msgs):
        # Extract msg timestamp
        if hasattr(msg, 'header'):
            msg_timestamp = round(msg.header.stamp.to_sec(), 6)
        else:  # topic == '/tf'
            msg_timestamp = round(msg.transforms[0].header.stamp.to_sec(), 6)

        if topic == '/tf' and msg.transforms[0].header.frame_id == "map" \
                and msg.transforms[0].child_frame_id == 'imu':
            frame_data['poses'].append(transform_msg2mat(msg.transforms[0].transform))
            frame_data['pose_timestamps'].append(msg_timestamp)
        elif topic == '/camera/color/image_raw':
            frame_data['rgb_imgs'].append(cvbridge.imgmsg_to_cv2(msg, desired_encoding='rgb8'))
            frame_data['rgb_img_timestamps'].append(msg_timestamp)
        elif topic == '/camera/color/camera_info':
            frame_data['rgb_intrinsics'].append(np.asanyarray(msg.P).reshape(3, 4)[:3, :3])
            frame_data['rgb_intr_timestamps'].append(msg_timestamp)
        elif topic == '/camera/aligned_depth_to_color/image_raw':
            frame_data['depth_imgs'].append(cvbridge.imgmsg_to_cv2(msg, desired_encoding='16UC1'))
            frame_data['depth_img_timestamps'].append(msg_timestamp)
        elif topic == '/camera/aligned_depth_to_color/camera_info':
            frame_data['depth_intrinsics'].append(np.asanyarray(msg.P).reshape(3, 4)[:3, :3])
            frame_data['depth_intr_timestamps'].append(msg_timestamp)

    # Find timestamps associations
    from functools import reduce
    timestamps = reduce(np.intersect1d, [v for k, v in frame_data.items() if '_timestamps' in k]).tolist()
    frame_data['timestamps'] = timestamps
    print(f"Found {len(timestamps)} timestamps with images and poses")
    
    return frame_data

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-05-30 12:56:12,111 - _logger - Log opened: Mon May 30 19:56:12 2022 UTC
INFO - 2022-05-30 12:56:12,145 - topics - topicmanager initialized


In [2]:
# dense reconstruction
bagpath = Path('/maya-slam/slam_algorithms/rosbags/d435i_bag_2022-05-20-22-23-07_posed.bag').resolve()
bag = rosbag.Bag(bagpath)

# Read frame data from rosbag
frame_data = read_frame_data_from_bag(bag)

Found 1488 timestamps with images and poses


In [3]:
frame_data.keys()

dict_keys(['rgb_imgs', 'rgb_img_timestamps', 'rgb_intrinsics', 'rgb_intr_timestamps', 'depth_imgs', 'depth_img_timestamps', 'depth_intrinsics', 'depth_intr_timestamps', 'poses', 'pose_timestamps', 'timestamps'])

In [4]:
import open3d.core as o3c

# Reference:
# http://www.open3d.org/docs/0.15.1/tutorial/t_reconstruction_system/voxel_block_grid.html

#device = o3c.Device('CUDA:0')
device = o3c.Device('CPU:0')

vbg = o3d.t.geometry.VoxelBlockGrid(
    attr_names=('tsdf', 'weight', 'color'),
    attr_dtypes=(o3c.float32, o3c.float32, o3c.float32),
    attr_channels=((1), (1), (3)),
    #voxel_size=3.0 / 512,  # ~5.86mm
    voxel_size=3.0 / 300,  # ~10mm
    block_resolution=16,
    block_count=50000,
    device=device
)

In [5]:
vbg

In [6]:
def get_frame_data_at_timestamp(frame_data: Dict[str, list], timestamp: float):
    """Get frame data at timestamp"""
    rgb_im = frame_data['rgb_imgs'][frame_data['rgb_img_timestamps'].index(timestamp)]
    rgb_K = frame_data['rgb_intrinsics'][frame_data['rgb_intr_timestamps'].index(timestamp)]
    
    depth_im = frame_data['depth_imgs'][frame_data['depth_img_timestamps'].index(timestamp)]
    depth_K = frame_data['depth_intrinsics'][frame_data['depth_intr_timestamps'].index(timestamp)]
    
    np.testing.assert_allclose(rgb_K, depth_K,
                               err_msg='Different intrinsic for rgb and depth, possibly misaligned')
    K = rgb_K
    
    cam_pose = frame_data['poses'][frame_data['pose_timestamps'].index(timestamp)]\
    
    return rgb_im, depth_im, K, cam_pose

In [7]:
from tqdm import tqdm
for i, timestamp in enumerate(tqdm(frame_data['timestamps'])):
    if i <= 100:
        continue
    
    rgb_im, depth_im, K, cam_pose = get_frame_data_at_timestamp(frame_data, timestamp)
    
    rgb_im = o3d.t.geometry.Image(rgb_im).to(device)
    depth_im = o3d.t.geometry.Image(depth_im).to(device)
    K = o3c.Tensor(K)
    extrinsic = o3c.Tensor(np.linalg.inv(cam_pose))
    
    frustum_block_coords = vbg.compute_unique_block_coordinates(
        depth_im, K, extrinsic, depth_scale=1e3, depth_max=10.0
    )
    
    vbg.integrate(frustum_block_coords, depth_im, rgb_im,
                  K, K, extrinsic, depth_scale=1e3, depth_max=10.0)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████| 1488/1488 [03:38<00:00,  6.82it/s]


In [8]:
pcd = vbg.extract_point_cloud()
pcd

PointCloud on CPU:0 [7028853 points (Float32)] Attributes: colors (dtype = Float32, shape = {7028853, 3}), normals (dtype = Float32, shape = {7028853, 3}).

In [10]:
o3d.visualization.draw_geometries([pcd.to_legacy()])

In [ ]:
# Kernel crash
mesh = vbg.extract_triangle_mesh()
mesh

In [ ]:
o3d.visualization.draw_geometries([mesh.to_legacy()])

In [10]:
o3d.io.write_triangle_mesh('test.ply', mesh)

NameError: name 'mesh' is not defined

In [14]:
o3d.io

<module 'open3d.cuda.pybind.io'>

In [13]:
o3d.cpu.pybind.io.write_triangle_mesh('test.ply', mesh)

AttributeError: module 'open3d' has no attribute 'cpu'

In [10]:
o3d.t.geometry.Image(depth_im).to(device)

Image[size={480,640}, channels=1, UInt16, CUDA:0]

In [18]:
o3c.Tensor(np.eye(4)).to(o3c.Device('CUDA:0'))

[[1.0 0.0 0.0 0.0],
 [0.0 1.0 0.0 0.0],
 [0.0 0.0 1.0 0.0],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CUDA:0, 0x7f65b69e8400]

In [19]:
o3c.det

<function open3d.cuda.pybind.core.PyCapsule.det>